<a href="https://colab.research.google.com/github/lqcgithub/MiningofMassiveDatasets/blob/main/TextSimilarityLSH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
pip install beautifulsoup4

In [26]:
pip install pyspark

In [27]:
pip install mmh3

In [28]:
import requests
from bs4 import BeautifulSoup

In [29]:
import pyspark
from pyspark import SparkConf, SparkContext

In [30]:
import re
import numpy as np
import collections
import random
from operator import itemgetter

In [31]:
import nltk
from nltk import ngrams
import mmh3
import time

In [32]:
conf = SparkConf().setMaster("local").setAppName("Market Basket with Apriori")
sc = SparkContext.getOrCreate(conf=conf)

In [33]:
baseUrl = "https://tuoitre.vn"
url = "https://tuoitre.vn/tin-moi-nhat.htm"

In [34]:
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

titles = soup.findAll('h3', class_='title-news')
links = [link.find('a').attrs["href"] for link in titles]
data = []

for link in links:
  news = requests.get(baseUrl + link)
  soup = BeautifulSoup(news.content, "html.parser")
  if (soup.find("h1", class_="article-title")is not None):
    title = soup.find("h1", class_="article-title").text
    abstract = soup.find("h2", class_="sapo").text
    body = soup.find("div", id="main-detail-body")
    contents = body.findAll("p", recursive=False)
    contents = [content.text for content in contents]
    content = ('').join(contents)
    data.append({
        "title": title,
        "abstract": abstract,
        "content": content,
    })

In [35]:
print(data[0]["content"])

Tùy vào trình độ bịp, các thiết bị này được người bán ra giá từ vài triệu đến hàng chục triệu đồng với cam kết "hàng" rất "an toàn", không gây hại đến sức khỏe người sử dụng! Tuy nhiên, người mua các thiết bị cờ bạc bịp này cũng có thể "ăn quả lừa" của người bán, chưa kể nguy cơ khi tiếp xúc các thiết bị này có thể chứa chất phóng xạ.Rao bán tràn lan trên mạngĐầu tháng 3, sau khi liên hệ với ông T., người đăng video trên mạng chào bán đồ chơi "xóc đĩa bịp - máy chụp X-quang nhìn xuyên đĩa", chúng tôi được hẹn đến cổng bến xe Miền Đông (TP.HCM) để đưa đi xem hàng. Tại điểm hẹn, chúng tôi được một nam thanh niên đi xe máy ra đón, dẫn về một chung cư trên đường Nguyễn Xí (quận Bình Thạnh).Sau khi ngồi đợi tại quán cà phê ở tầng trệt chung cư như yêu cầu, chúng tôi được một thanh niên khác dẫn lên căn hộ ở tầng 21. Một thanh niên lấy ra thiết bị màu đen nhỏ bằng ngón tay và giới thiệu đây là máy chơi cờ bạc bịp mới nhất, giá 7 triệu đồng, có thể báo (rung) cho người chơi kết quả chính xác 

In [36]:
def preprocess(document):
    document = re.sub(r'[^\w\s]','',document)
    tokens = document.lower()
    tokens = tokens.split()
    return tokens

In [37]:
document = "Trận này, Leicester áp đảo ngay sau tiếng còi khai cuộc. Đây cũng là điều dễ hiểu, bởi đối thủ của họ - Sheffield United là đội bóng đang xếp cuối bảng và gần như cầm chắc vé xuống hạng khi kém nhóm an toàn tới 14 điểm.Phút 12, cơ hội mở tỉ số đến với Leicester sau pha dàn xếp tấn công bên cánh trái. Nhưng may mắn đã không đứng về phía đội chủ nhà khi pha đánh đầu về góc xa của Ayoze Perez lại đưa bóng trúng cột dọc khung thành.  Những phút sau đó, Leicester tiếp tục tấn công và tạo ra thêm nhiều tình huống nguy hiểm nữa. Dù áp đảo hoàn toàn, nhưng phải đến cuối hiệp 1 các học trò HLV Brendan Rodgers mới đưa được bóng vào lưới khung thành thủ môn Aaron Ramsdale.  Phút 39, từ pha lên bóng bên cánh trái, Jamie Vardy bứt tốc xâm nhập vòng cấm đón đường chọc khe của đồng đội rồi thực hiện đường căng ngang vào giữa. Trong tư thế không người kèm, Kelechi Iheanacho dễ dàng ập vào đệm bóng tung lưới thủ môn Aaron Ramsdale từ cự li gần, mở tỉ số cho Leicester.  Bị thủng lưới, Sheffield United đẩy cao đội hình tấn công trong hiệp 2 và cơ hội tiếp tục đến với Leicester khi hàng phòng ngự đội khách để lộ ra nhiều khoảng trống.  Phút 64, sau vài cơ hội bị bỏ lỡ, đội chủ nhà cũng đã có bàn thắng nhân đôi cách biệt nhờ công tiền vệ Ayoze Perez với pha dứt điểm hiểm hóc ngay mép vòng cấm sau tình huống phản công bên cánh phải.  Sau bàn thua này, các cầu thủ Sheffield United càng thi đấu thiếu tập trung, đặc biệt là trong phòng ngự.  Hệ quả, chỉ trong 11 phút từ 69 đến 80, lưới của thủ môn Aaron Ramsdale đã phải rung lên thêm 3 lần nữa sau những cú dứt điểm trong tư thế vô cùng thoải mái của Kelechi Iheanacho (phút 69 và 78) và Jamie Vardy (phút 80), giúp Leicester khép lại trận đấu với chiến thắng cách biệt 5-0.  Thắng trận này, Leicester đã tạm vượt qua Manchester United (chưa đá vòng 28) để vươn lên vị trí thứ hai trên bảng xếp hạng với 56 điểm, hơn Man Utd 2 điểm và kém đội đầu bảng Man City 15 điểm. "
preprocess(document)[0:5]

['trận', 'này', 'leicester', 'áp', 'đảo']

In [38]:
doc_as_shingles = []
for document in data:
  tokens = preprocess(document['content'])
  shingles = ngrams(tokens, 3)
  doc_as_shingles.append(list(shingles))
print(doc_as_shingles[0])

[('tùy', 'vào', 'trình'), ('vào', 'trình', 'độ'), ('trình', 'độ', 'bịp'), ('độ', 'bịp', 'các'), ('bịp', 'các', 'thiết'), ('các', 'thiết', 'bị'), ('thiết', 'bị', 'này'), ('bị', 'này', 'được'), ('này', 'được', 'người'), ('được', 'người', 'bán'), ('người', 'bán', 'ra'), ('bán', 'ra', 'giá'), ('ra', 'giá', 'từ'), ('giá', 'từ', 'vài'), ('từ', 'vài', 'triệu'), ('vài', 'triệu', 'đến'), ('triệu', 'đến', 'hàng'), ('đến', 'hàng', 'chục'), ('hàng', 'chục', 'triệu'), ('chục', 'triệu', 'đồng'), ('triệu', 'đồng', 'với'), ('đồng', 'với', 'cam'), ('với', 'cam', 'kết'), ('cam', 'kết', 'hàng'), ('kết', 'hàng', 'rất'), ('hàng', 'rất', 'an'), ('rất', 'an', 'toàn'), ('an', 'toàn', 'không'), ('toàn', 'không', 'gây'), ('không', 'gây', 'hại'), ('gây', 'hại', 'đến'), ('hại', 'đến', 'sức'), ('đến', 'sức', 'khỏe'), ('sức', 'khỏe', 'người'), ('khỏe', 'người', 'sử'), ('người', 'sử', 'dụng'), ('sử', 'dụng', 'tuy'), ('dụng', 'tuy', 'nhiên'), ('tuy', 'nhiên', 'người'), ('nhiên', 'người', 'mua'), ('người', 'mua', 'các

In [39]:
doc_hashed = []

def hash_doc(shingles):
  shingles_hashed = []
  for shingle in shingles:
    hash = mmh3.hash(' '.join(shingle))
    shingles_hashed.append(hash)
  return shingles_hashed  

for document in doc_as_shingles:
  doc_hashed.append(list(hash_doc(document)));
print(doc_hashed[0])

[518581185, 456688079, 275791069, -422903667, 584949940, -519954334, 1481381068, -655941761, 1568172651, 1088287363, -848649469, 145809575, -214779842, 1573778994, -897480541, -1575088200, -834580802, 1043844664, -1537029789, -39106991, 135752693, -1331088568, 65492383, 161069262, 651135317, 560162720, -1704458593, 1968178039, 803440172, 1659701131, -318850459, 29543772, -1280394785, 469529709, -1791196905, -116086071, 1616351483, -887850927, 1226251224, -218342886, -1354060775, 446203753, -519954334, 465329231, 514223004, 1394147596, 2118211547, 1155839214, 25771153, -7102387, -914859816, 1907181265, 310568618, -858553368, -1812562404, -61516475, 97729598, 184881738, 1257103788, -1612397406, -1321222764, 24232248, -1715364537, -340235686, -1246277415, -519954334, 1481381068, -1540664259, 492286502, -1745507127, 622450652, -912027018, 1506110984, -838536566, -1388808785, -609253119, -412337517, -1323853938, 1131894107, 1270965241, -344591680, -1871512705, -765467060, -856604693, 202751

In [40]:
start = time.time()
signatures = []
max_shingle_id = 2 ** 32 - 1

next_prime = 4294967311

def pick_random_coeffs(k):
    rand_list = []

    while k > 0:
        rand_index = random.randint(0, max_shingle_id)

        while rand_index in rand_list:
            rand_index = random.randint(0, max_shingle_id)

        rand_list.append(rand_index)
        k = k - 1
    return rand_list

coeff_a = pick_random_coeffs(10)
coeff_b = pick_random_coeffs(10)

for document in doc_hashed:
  signature =[]
  for i in range(0, 10):
    min_hash_code = next_prime + 1
    for shingle in document: 
      hash_code = (coeff_a[i] * shingle + coeff_b[i]) % next_prime
      if hash_code < min_hash_code:
        min_hash_code = hash_code
    signature.append(min_hash_code)
  signatures.append(signature)
end = time.time()
print(signatures[0])
print(len(signatures),len(signatures[0]))
print("Minhash completed in : {:.4f} sec".format(end - start))

[6672529, 4902159, 29695954, 1408885, 7256176, 22676914, 2159337, 36078635, 236852, 953213]
20 10
Minhash completed in : 0.0861 sec


In [41]:
def jaccardSim(d1,d2):
    return len(d1.intersection(d2))/len(d1.union(d2))   

In [42]:
sims = jaccardSim(set(signatures[0]), set(signatures[1])) 
sims

0.0

In [43]:
def initialize_array_bucket(bands):
    len_buckets = 101
    array_buckets = []
    for band in range(bands):
        array_buckets.append([[] for i in range(len_buckets)])
    return array_buckets

In [47]:
def apply_LSH_technique(signatures,t,bands,rows):
    if bands * rows != len(signatures):
        print('split to bands error.')

    array_buckets = initialize_array_bucket(bands)

    # hash_funcs = pick_random_coeffs(10);


    candidates = {}
    
    i = 0
    for b in range(bands):
        buckets = array_buckets[b]        
        band = np.array(signatures)[i:i+rows,:]
        for col in range(band.shape[1]):    
            
            key = int(sum(band[:,col]) % len(buckets))
            
            buckets[key].append(col)
        i = i+rows

        for item in buckets:
            if len(item) > 1:
                pair = (item[0], item[1])
                if pair not in candidates:
                    A = np.array(signatures)[:,item[0]]
                    B = np.array(signatures)[:,item[1]]
                    similarity = cosine_distance(A,B)
                    if similarity >= t:
                        candidates[pair] = similarity


    sort = sorted(candidates.items(),key=itemgetter(1), reverse=True)

    return candidates,sort

def euclidean_distance(x,y,r=2.0):
    try:
        
        return sum(((x[i] - y[i]) ** r) for i in range(len(x))) ** (1.0/r)
    
    except (ValueError,ZeroDivisionError):
        print ('Please, enter only even values for "r > 0".')
    except IndexError:
        print ('Please, the sets must have the same size.')

def cosine_distance(x,y):
    prodAB = sum([x[i]*y[i] for i in range(len(x))])
    zeros = [0 for i in range(len(x))]
    A = euclidean_distance(x,zeros)
    B = euclidean_distance(y,zeros)
    return prodAB / (A*B)

start = time.time()
candidates, sort = apply_LSH_technique(signatures, 0.7, 4, 5)
end = time.time()
print(candidates)
print("LSH completed in: {:.4f}".format(end - start))

{}
LSH completed in: 0.0013


In [77]:
# baseUrl = "https://kenh14.vn/"
# url = "https://kenh14.vn/musik.chn"
# def crawNewsData(baseUrl, url):
#     response = requests.get(url)
#     soup = BeautifulSoup(response.content, "html.parser")
#     regex = re.compile("*-title")
#     titles = soup.findAll('h3', class_=lamda x : re.match(regex, x))
#     print(titles)

#     links = [link.find('a').attrs["href"] for link in titles]
#     data = []
#     for link in links:
#         news = requests.get(baseUrl + link)
#         soup = BeautifulSoup(news.content, "html.parser")
        
#         # title =soup.find("div", class_=["*-title"]}, partial=True)
#         print(title)
#         # abstract = soup.find("div", "class"_="sapo").text
#         # body = soup.find("div", {"class": "-content"})
#         content = ""
#         try:
#             contents = body.findAll("p", recursive=False)
#             contents = [content.text for content in contents]
#             content = ('').join(contents)
#         except:
#             content = ""
#         data.append({
#             "title": title,
#             "abstract": abstract,
#             "content": content,
#         })
#     return data
# data = crawNewsData(baseUrl, url)


SyntaxError: ignored